# Trend Following Backtest - Local Data Version

Uses pre-downloaded continuous futures data instead of live Norgate calls.

**Data paths:**
- Price data: `./02-futures_prices/norgate_continuous/csv/` or `/parquet/`
- Margins: `margin_requirements.csv`

In [1]:
import pandas as pd
import numpy as np
import warnings
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass
from datetime import datetime
from pathlib import Path
import json
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (14, 6)

# Optional Norgate for metadata only
try:
    import norgatedata
    NORGATE_AVAILABLE = norgatedata.status()
except:
    NORGATE_AVAILABLE = False

print(f"✓ Imports successful | Norgate metadata: {NORGATE_AVAILABLE}")

[2026-01-21 19:17:17.243345] INFO: Norgate Data: NorgateData package v1.0.74: Init complete
✓ Imports successful | Norgate metadata: True


In [7]:
# ==================== CONFIGURATION ====================
CONFIG = {
    'initial_capital': 400000,
    'commission_per_contract': 2.50,
    'fast_period': 50,
    'slow_period': 100,
    'vol_lookback': 60,
    'target_portfolio_vol': 0.20,
    'risk_per_trade': 0.01,
    'target_margin_usage': 0.25,
    'max_leverage': 3.0,
    'start_date': '2015-01-01',
    'end_date': None,
    
    # LOCAL DATA PATHS - adjust these to match your setup
    'csv_dir': './02-futures_prices/norgate_continuous/csv',
    'parquet_dir': './02-futures_prices/norgate_continuous/parquet',
    'prefer_parquet': True,
    'margin_file_path': './External_margin_file/margin_requirements.csv',
    'margin_column': 'Overnight Maintenance',
}

print("Configuration loaded")
for k, v in CONFIG.items():
    print(f"  {k}: {v}")

Configuration loaded
  initial_capital: 400000
  commission_per_contract: 2.5
  fast_period: 50
  slow_period: 100
  vol_lookback: 60
  target_portfolio_vol: 0.2
  risk_per_trade: 0.01
  target_margin_usage: 0.25
  max_leverage: 3.0
  start_date: 2015-01-01
  end_date: None
  csv_dir: ./02-futures_prices/norgate_continuous/csv
  parquet_dir: ./02-futures_prices/norgate_continuous/parquet
  prefer_parquet: True
  margin_file_path: ./External_margin_file/margin_requirements.csv
  margin_column: Overnight Maintenance


In [8]:
# ==================== DATA CLASSES & METADATA ====================
@dataclass
class FuturesContract:
    symbol: str
    name: str
    sector: str
    point_value: float
    tick_size: float
    margin_requirement: float
    margin_source: str = 'unknown'

# Fallback metadata when Norgate unavailable
FALLBACK_POINT_VALUES = {
    'ES': 50, 'MES': 5, 'NQ': 20, 'RTY': 50, 'YM': 5, 'EMD': 100,
    'FESX': 10, 'FDAX': 25, 'NIY': 5, 'NKD': 5,
    'ZB': 1000, 'ZN': 1000, 'ZF': 1000, 'ZT': 2000,
    'FGBL': 1000, 'FGBM': 1000, 'FGBS': 1000,
    'CL': 1000, 'NG': 10000, 'HO': 42000, 'RB': 42000, 'BRN': 1000,
    'GC': 100, 'SI': 5000, 'HG': 25000, 'PL': 50, 'PA': 100,
    'ZC': 50, 'ZS': 50, 'ZW': 50, 'ZM': 100, 'ZL': 600,
    'CT': 500, 'KC': 375, 'SB': 1120, 'CC': 10,
    '6E': 125000, '6J': 12500000, '6B': 62500, '6A': 100000,
    '6C': 100000, '6S': 125000, 'DX': 1000,
    'VX': 1000,
}

FALLBACK_SECTORS = {
    'ES': 'Stock Index', 'MES': 'Stock Index', 'NQ': 'Stock Index', 'RTY': 'Stock Index',
    'YM': 'Stock Index', 'EMD': 'Stock Index', 'FESX': 'Stock Index', 'FDAX': 'Stock Index',
    'NIY': 'Stock Index', 'NKD': 'Stock Index',
    'ZB': 'Interest Rate', 'ZN': 'Interest Rate', 'ZF': 'Interest Rate', 'ZT': 'Interest Rate',
    'FGBL': 'Interest Rate', 'FGBM': 'Interest Rate', 'FGBS': 'Interest Rate',
    'CL': 'Energy', 'NG': 'Energy', 'HO': 'Energy', 'RB': 'Energy', 'BRN': 'Energy',
    'GC': 'Metals', 'SI': 'Metals', 'HG': 'Metals', 'PL': 'Metals', 'PA': 'Metals',
    'ZC': 'Agriculture', 'ZS': 'Agriculture', 'ZW': 'Agriculture', 'ZM': 'Agriculture',
    'ZL': 'Agriculture', 'CT': 'Agriculture', 'KC': 'Agriculture', 'SB': 'Agriculture', 'CC': 'Agriculture',
    '6E': 'Currency', '6J': 'Currency', '6B': 'Currency', '6A': 'Currency',
    '6C': 'Currency', '6S': 'Currency', 'DX': 'Currency',
    'VX': 'Volatility',
}

print("✓ Data classes defined")

✓ Data classes defined


In [9]:
# ==================== LOCAL DATA LOADING ====================
def load_local_price_data(symbol: str) -> Optional[pd.DataFrame]:
    """Load price data from local CSV/Parquet files."""
    clean_symbol = symbol.strip().upper().lstrip('&')
    
    if CONFIG['prefer_parquet']:
        paths = [
            (CONFIG['parquet_dir'], f"{clean_symbol}.parquet", 'parquet'),
            (CONFIG['csv_dir'], f"{clean_symbol}.csv", 'csv'),
        ]
    else:
        paths = [
            (CONFIG['csv_dir'], f"{clean_symbol}.csv", 'csv'),
            (CONFIG['parquet_dir'], f"{clean_symbol}.parquet", 'parquet'),
        ]
    
    for directory, filename, fmt in paths:
        filepath = Path(directory) / filename
        if filepath.exists():
            try:
                if fmt == 'parquet':
                    df = pd.read_parquet(filepath)
                else:
                    df = pd.read_csv(filepath)
                return _standardize_df(df)
            except Exception as e:
                continue
    return None

def _standardize_df(df: pd.DataFrame) -> pd.DataFrame:
    """Standardize column names: lowercase -> Title Case"""
    if 'date' in df.columns:
        df = df.set_index('date')
    if not isinstance(df.index, pd.DatetimeIndex):
        df.index = pd.to_datetime(df.index)
    df = df.sort_index()
    
    col_map = {'open': 'Open', 'high': 'High', 'low': 'Low', 
               'close': 'Close', 'volume': 'Volume', 'settle': 'Close'}
    df.columns = [col_map.get(c.lower().strip(), c) for c in df.columns]
    return df

def get_available_symbols() -> List[str]:
    """Get symbols with local data."""
    symbols = set()
    for d in [CONFIG['csv_dir'], CONFIG['parquet_dir']]:
        p = Path(d)
        if p.exists():
            for f in p.glob('*.*'):
                if f.suffix in ['.csv', '.parquet']:
                    symbols.add(f.stem.upper())
    return sorted(symbols)

# Test
available = get_available_symbols()
print(f"✓ Found {len(available)} symbols with local data")
if available:
    print(f"  First 10: {available[:10]}")

✓ Found 105 symbols with local data
  First 10: ['6A', '6B', '6C', '6E', '6J', '6M', '6N', '6S', 'AFB', 'AWM']


In [10]:
# ==================== MARGIN & METADATA ====================
def load_margin_lookup() -> Dict:
    """Load margin requirements from CSV."""
    try:
        df = pd.read_csv(CONFIG['margin_file_path'])
        print(f"✓ Loaded {len(df)} margin records")
    except FileNotFoundError:
        print(f"⚠ Margin file not found")
        return {}
    
    lookup = {}
    if 'Currency' in df.columns:
        df = df.sort_values(by='Currency', key=lambda x: x != 'USD')
    
    for _, row in df.iterrows():
        sym = str(row.get('Trading Class', '')).strip().upper()
        if not sym or sym in lookup:
            continue
        try:
            margin = float(row[CONFIG['margin_column']])
            if pd.notna(margin) and margin > 0:
                lookup[sym] = margin
        except:
            continue
    print(f"✓ {len(lookup)} unique margin entries")
    return lookup

def get_margin(symbol: str, lookup: Dict) -> Tuple[float, str]:
    """Get margin for symbol."""
    sym = symbol.strip().upper().lstrip('&')
    mappings = {'AD': '6A', 'BP': '6B', 'CD': '6C', 'EC': '6E', 'JY': '6J', 'SF': '6S',
                'C': 'ZC', 'W': 'ZW', 'S': 'ZS', 'SM': 'ZM', 'BO': 'ZL',
                'US': 'ZB', 'TY': 'ZN', 'FV': 'ZF', 'TU': 'ZT'}
    
    for s in [sym, mappings.get(sym)]:
        if s and s in lookup:
            return lookup[s], 'spreadsheet'
    
    if NORGATE_AVAILABLE:
        try:
            m = norgatedata.margin(f"&{sym}")
            if m and m > 0:
                return m, 'norgate'
        except:
            pass
    return 5000, 'default'

def get_metadata(symbol: str) -> Tuple[str, float, float, str]:
    """Get name, point_value, tick_size, sector."""
    sym = symbol.strip().upper().lstrip('&')
    name = sym
    pv = FALLBACK_POINT_VALUES.get(sym, 1000)
    ts = 0.01
    sector = FALLBACK_SECTORS.get(sym, 'Unknown')
    
    if NORGATE_AVAILABLE:
        try: name = norgatedata.futures_market_name(sym) or sym
        except: pass
        try:
            v = norgatedata.point_value(f"&{sym}")
            if v and v > 0: pv = v
        except: pass
        try:
            v = norgatedata.tick_size(f"&{sym}")
            if v and v > 0: ts = v
        except: pass
        try:
            v = norgatedata.classification(f"&{sym}", 'NorgateDataFuturesClassification', 'Name')
            if v: sector = v
        except: pass
    return name, pv, ts, sector

margin_lookup = load_margin_lookup()

✓ Loaded 355 margin records
✓ 349 unique margin entries


In [11]:
# ==================== BUILD INSTRUMENT UNIVERSE ====================
SELECTED_SYMBOLS = [
    'MES', 'NQ', 'RTY', 'YM', 'EMD', 'FESX', 'FDAX', 'NIY', 'NKD',
    'ZB', 'ZN', 'ZF', 'ZT', 'FGBL', 'FGBM', 'FGBS',
    'CL', 'NG', 'HO', 'RB', 'BRN',
    'GC', 'SI', 'HG', 'PL', 'PA',
    'ZC', 'ZS', 'ZW', 'ZM', 'ZL', 'CT', 'KC', 'SB', 'CC',
    '6E', '6J', '6B', '6A', '6C', '6S', 'DX',
    'VX',
]

def build_universe(symbols: List[str] = None, max_margin: float = None) -> List[FuturesContract]:
    """Build instrument universe from local data."""
    symbols = symbols or SELECTED_SYMBOLS
    available = set(get_available_symbols())
    instruments = []
    skipped = []
    
    for sym in symbols:
        s = sym.strip().upper()
        if s not in available:
            skipped.append(f"{s} (no local data)")
            continue
        
        name, pv, ts, sector = get_metadata(s)
        margin, src = get_margin(s, margin_lookup)
        
        if max_margin and margin > max_margin:
            skipped.append(f"{s} (margin ${margin:,.0f})")
            continue
        
        instruments.append(FuturesContract(s, name, sector, pv, ts, margin, src))
    
    print(f"✓ Universe: {len(instruments)} instruments")
    if skipped:
        print(f"⚠ Skipped {len(skipped)}: {skipped[:5]}..." if len(skipped) > 5 else f"⚠ Skipped: {skipped}")
    return instruments

instruments = build_universe(max_margin=CONFIG['initial_capital'] * 0.05)

✓ Universe: 34 instruments
⚠ Skipped 9: ['NQ (margin $33,695)', 'EMD (margin $23,038)', 'FDAX (margin $45,483)', 'NIY (margin $1,769,900)', 'HO (margin $20,160)']...


In [12]:
# Display universe
print(f"\n{'='*90}")
print(f"INSTRUMENT UNIVERSE - {len(instruments)} CONTRACTS")
print(f"{'='*90}")

by_sector = {}
for inst in instruments:
    by_sector.setdefault(inst.sector, []).append(inst)

for sector in sorted(by_sector):
    insts = by_sector[sector]
    print(f"\n{sector}: {len(insts)}")
    for i in insts:
        print(f"   {i.symbol:6} {i.name[:22]:23} PV:${i.point_value:>7,.0f}  Margin:${i.margin_requirement:>7,.0f} ({i.margin_source})")


INSTRUMENT UNIVERSE - 34 CONTRACTS

Agriculture & Livestock: 8
   ZC     Corn                    PV:$     50  Margin:$  1,416 (spreadsheet)
   ZS     Soybean                 PV:$     50  Margin:$  3,216 (spreadsheet)
   ZW     Chicago SRW Wheat       PV:$     50  Margin:$  1,757 (spreadsheet)
   ZM     Soybean Meal            PV:$    100  Margin:$  2,438 (spreadsheet)
   ZL     Soybean Oil             PV:$    600  Margin:$  2,919 (spreadsheet)
   CT     Cotton No. 2            PV:$    500  Margin:$  1,467 (spreadsheet)
   SB     Sugar No. 11            PV:$  1,120  Margin:$  1,451 (spreadsheet)
   CC     Cocoa                   PV:$     10  Margin:$  9,736 (spreadsheet)

Currency: 7
   6E     Euro FX                 PV:$125,000  Margin:$  2,819 (spreadsheet)
   6J     Japanese Yen            PV:$125,000  Margin:$  2,800 (spreadsheet)
   6B     British Pound           PV:$ 62,500  Margin:$  2,094 (spreadsheet)
   6A     Australian Dollar       PV:$100,000  Margin:$  1,943 (spreadsheet)

In [13]:
# ==================== STRATEGY CLASSES ====================
class TrendFollowingStrategy:
    def __init__(self, fast=50, slow=100, vol_lb=60):
        self.fast_period, self.slow_period, self.vol_lookback = fast, slow, vol_lb
    
    def calculate_signals(self, prices: pd.Series) -> pd.Series:
        fast = prices.rolling(self.fast_period).mean()
        slow = prices.rolling(self.slow_period).mean()
        sig = pd.Series(0, index=prices.index)
        sig[fast > slow] = 1
        sig[fast < slow] = -1
        return sig
    
    def calculate_volatility(self, prices: pd.Series) -> pd.Series:
        return prices.pct_change().rolling(self.vol_lookback).std() * np.sqrt(252)

class PositionSizer:
    def __init__(self, account, target_vol=0.20, risk=0.01, max_lev=3.0):
        self.account_size = account
        self.risk_per_trade = risk
        self.max_leverage = max_lev
    
    def calculate_position_size(self, signal, inst, price, vol, _) -> int:
        if signal == 0 or np.isnan(vol) or vol == 0:
            return 0
        dollar_vol = price * inst.point_value * vol
        contracts = int(np.sign(signal) * self.account_size * self.risk_per_trade / dollar_vol)
        # Leverage limit
        notional = abs(contracts) * price * inst.point_value
        if notional > self.account_size * self.max_leverage:
            contracts = int(np.sign(contracts) * self.account_size * self.max_leverage / (price * inst.point_value))
        return contracts

class PortfolioManager:
    def __init__(self, account, target_margin=0.25):
        self.account_size = account
        self.target_margin_usage = target_margin
    
    def update_positions(self, positions):
        total_margin = sum(abs(c) * i.margin_requirement for c, i in positions.values())
        max_margin = self.account_size * self.target_margin_usage
        if total_margin > max_margin:
            scale = max_margin / total_margin
            return {s: (int(c * scale), i) for s, (c, i) in positions.items()}
        return positions
    
    def get_portfolio_statistics(self, positions):
        total_margin = sum(abs(c) * i.margin_requirement for c, i, _ in positions.values())
        return {
            'margin_utilization': total_margin / self.account_size if self.account_size > 0 else 0,
            'num_positions': len([c for c, _, _ in positions.values() if c != 0])
        }

print("✓ Strategy classes defined")

✓ Strategy classes defined


In [14]:
# ==================== BACKTEST ENGINE ====================
class LocalDataBacktestEngine:
    def __init__(self, instruments, account=400000, fast=50, slow=100,
                 target_vol=0.20, risk=0.01, target_margin=0.25, commission=2.50):
        self.initial_capital = account
        self.commission = commission
        self.strategy = TrendFollowingStrategy(fast, slow)
        self.position_sizer = PositionSizer(account, target_vol, risk)
        self.portfolio_manager = PortfolioManager(account, target_margin)
        self.instruments = {i.symbol: i for i in instruments}
    
    def load_data(self, start, end=None):
        """Load local price data."""
        print(f"Loading data ({start} to {end or 'latest'})...")
        start_dt = pd.Timestamp(start)
        end_dt = pd.Timestamp(end) if end else None
        
        data = {}
        for sym in self.instruments:
            df = load_local_price_data(sym)
            if df is not None:
                df = df[df.index >= start_dt]
                if end_dt:
                    df = df[df.index <= end_dt]
                if len(df) > 0:
                    data[sym] = df
                    print(f"  {sym}: {len(df)} records")
        print(f"✓ Loaded {len(data)}/{len(self.instruments)} instruments")
        return data
    
    def run_backtest(self, price_data):
        """Run backtest."""
        # Common dates
        all_dates = None
        for df in price_data.values():
            all_dates = set(df.index) if all_dates is None else all_dates & set(df.index)
        dates = sorted(all_dates)
        
        print(f"Backtest: {dates[0].date()} to {dates[-1].date()} ({len(dates)} days)")
        
        positions = {s: 0 for s in self.instruments}
        equity = self.initial_capital
        results = []
        
        for i, date in enumerate(dates):
            if i < self.strategy.slow_period:
                results.append({'date': date, 'equity': equity, 'daily_return': 0,
                               'num_positions': 0, 'margin_usage': 0})
                continue
            
            new_pos = {}
            for sym, inst in self.instruments.items():
                if sym not in price_data or date not in price_data[sym].index:
                    continue
                df = price_data[sym]
                hist = df.loc[df.index <= date, 'Close']
                price = hist.iloc[-1]
                sig = self.strategy.calculate_signals(hist).iloc[-1]
                vol = self.strategy.calculate_volatility(hist).iloc[-1]
                contracts = self.position_sizer.calculate_position_size(sig, inst, price, vol, positions)
                new_pos[sym] = (contracts, inst, price)
            
            # Apply margin constraints
            managed = self.portfolio_manager.update_positions(
                {s: (c, i) for s, (c, i, _) in new_pos.items()})
            
            # Calculate P&L
            daily_pnl = 0
            comm = sum(abs(managed.get(s, (0,))[0] - positions[s]) * self.commission
                      for s in self.instruments if s in price_data and date in price_data[s].index)
            
            if i > 0:
                prev = dates[i-1]
                for sym, c in positions.items():
                    if c != 0 and sym in price_data:
                        df = price_data[sym]
                        if date in df.index and prev in df.index:
                            daily_pnl += c * (df.loc[date, 'Close'] - df.loc[prev, 'Close']) * self.instruments[sym].point_value
            
            equity += daily_pnl - comm
            
            # Update positions
            for s, (c, _) in managed.items():
                positions[s] = c
            
            pos_data = {s: (positions[s], self.instruments[s], new_pos.get(s, (0,0,0))[2])
                       for s in positions}
            stats = self.portfolio_manager.get_portfolio_statistics(pos_data)
            
            results.append({'date': date, 'equity': equity,
                           'daily_return': (daily_pnl - comm) / (equity - daily_pnl + comm) if equity > 0 else 0,
                           'num_positions': stats['num_positions'],
                           'margin_usage': stats['margin_utilization']})
            
            self.position_sizer.account_size = equity
            self.portfolio_manager.account_size = equity
            
            if i % 500 == 0:
                print(f"  Day {i}/{len(dates)}...")
        
        return pd.DataFrame(results).set_index('date')
    
    def calculate_metrics(self, results):
        """Calculate performance metrics."""
        ret = results['daily_return']
        eq = results['equity']
        
        total_ret = (eq.iloc[-1] - self.initial_capital) / self.initial_capital
        years = len(ret) / 252
        ann_ret = (1 + total_ret) ** (1 / years) - 1 if years > 0 else 0
        ann_vol = ret.std() * np.sqrt(252)
        sharpe = ann_ret / ann_vol if ann_vol > 0 else 0
        
        cum = (1 + ret).cumprod()
        dd = (cum - cum.expanding().max()) / cum.expanding().max()
        max_dd = dd.min()
        calmar = ann_ret / abs(max_dd) if max_dd != 0 else 0
        
        wins = (ret > 0).sum()
        losses = (ret < 0).sum()
        
        return {
            'Total Return (%)': total_ret * 100,
            'Ann Return (%)': ann_ret * 100,
            'Ann Vol (%)': ann_vol * 100,
            'Sharpe': sharpe,
            'Max DD (%)': max_dd * 100,
            'Calmar': calmar,
            'Win Rate (%)': wins / (wins + losses) * 100 if (wins + losses) > 0 else 0,
            'Final Equity': eq.iloc[-1],
            'Avg Margin (%)': results['margin_usage'].mean() * 100,
        }

print("✓ Backtest engine defined")

✓ Backtest engine defined


In [15]:
# ==================== RUN BACKTEST ====================
engine = LocalDataBacktestEngine(
    instruments=instruments,
    account=CONFIG['initial_capital'],
    fast=CONFIG['fast_period'],
    slow=CONFIG['slow_period'],
    target_vol=CONFIG['target_portfolio_vol'],
    risk=CONFIG['risk_per_trade'],
    target_margin=CONFIG['target_margin_usage'],
    commission=CONFIG['commission_per_contract']
)

price_data = engine.load_data(CONFIG['start_date'], CONFIG['end_date'])

Loading data (2015-01-01 to latest)...
  MES: 502 records
  RTY: 502 records
  YM: 502 records
  FESX: 506 records
  NKD: 502 records
  ZB: 502 records
  ZN: 502 records
  ZF: 502 records
  ZT: 502 records
  FGBL: 506 records
  FGBM: 506 records
  FGBS: 506 records
  CL: 502 records
  NG: 502 records
  RB: 502 records
  BRN: 516 records
  HG: 502 records
  PL: 502 records
  ZC: 502 records
  ZS: 502 records
  ZW: 502 records
  ZM: 502 records
  ZL: 502 records
  CT: 502 records
  SB: 502 records
  CC: 502 records
  6E: 502 records
  6J: 502 records
  6B: 502 records
  6A: 502 records
  6C: 502 records
  6S: 502 records
  DX: 516 records
  VX: 502 records
✓ Loaded 34/34 instruments


In [16]:
print("\nRunning backtest...")
results = engine.run_backtest(price_data)
metrics = engine.calculate_metrics(results)
print(f"\n✓ Complete! Final equity: ${metrics['Final Equity']:,.2f}")


Running backtest...
Backtest: 2024-01-22 to 2026-01-20 (492 days)

✓ Complete! Final equity: $397,425.79


In [17]:
# ==================== PERFORMANCE REPORT ====================
print("=" * 60)
print("PERFORMANCE METRICS")
print("=" * 60)
for k, v in metrics.items():
    if 'Equity' in k:
        print(f"  {k:20} ${v:>15,.2f}")
    elif '%' in k:
        print(f"  {k:20} {v:>15.2f}%")
    else:
        print(f"  {k:20} {v:>15.3f}")

PERFORMANCE METRICS
  Total Return (%)               -0.64%
  Ann Return (%)                 -0.33%
  Ann Vol (%)                     1.86%
  Sharpe                        -0.178
  Max DD (%)                     -2.50%
  Calmar                        -0.132
  Win Rate (%)                   50.39%
  Final Equity         $     397,425.79
  Avg Margin (%)                  1.21%


In [ ]:
# ==================== VISUALIZATION ====================
fig, axes = plt.subplots(3, 1, figsize=(14, 12))

# Equity curve
ax1 = axes[0]
ax1.plot(results.index, results['equity'], linewidth=1.5, color='blue')
ax1.axhline(CONFIG['initial_capital'], color='gray', linestyle='--', alpha=0.5)
ax1.set_title('Equity Curve', fontsize=14, fontweight='bold')
ax1.set_ylabel('Equity ($)')
ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))
ax1.grid(True, alpha=0.3)

# Drawdown
ax2 = axes[1]
cum = (1 + results['daily_return']).cumprod()
dd = (cum - cum.expanding().max()) / cum.expanding().max() * 100
ax2.fill_between(results.index, 0, dd, alpha=0.5, color='red')
ax2.set_title('Drawdown', fontsize=14, fontweight='bold')
ax2.set_ylabel('Drawdown (%)')
ax2.grid(True, alpha=0.3)

# Margin
ax3 = axes[2]
ax3.plot(results.index, results['margin_usage'] * 100, linewidth=1, color='purple', alpha=0.7)
ax3.axhline(CONFIG['target_margin_usage'] * 100, color='red', linestyle='--', label='Target')
ax3.fill_between(results.index, 0, results['margin_usage'] * 100, alpha=0.3, color='purple')
ax3.set_title('Margin Utilization', fontsize=14, fontweight='bold')
ax3.set_ylabel('Margin (%)')
ax3.legend()
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('backtest_results.png', dpi=150, bbox_inches='tight')
plt.show()
print("✓ Charts saved to backtest_results.png")

In [ ]:
# ==================== SAVE RESULTS ====================
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

results.to_csv(f"backtest_results_{timestamp}.csv")
print(f"✓ Results saved: backtest_results_{timestamp}.csv")

with open(f"metrics_{timestamp}.json", 'w') as f:
    json.dump({k: float(v) if isinstance(v, (np.floating, np.integer)) else v 
               for k, v in metrics.items()}, f, indent=2)
print(f"✓ Metrics saved: metrics_{timestamp}.json")